In [ ]:

import re
import nltk
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# Define the label mapping
label_mapping = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

# Text normalization function
def normalize(text):
    # Remove newline characters
    text = text.replace("\n", " ").replace("\r", " ")
    # Remove punctuation and non-word characters (except whitespace)
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Trim leading and trailing whitespace
    text = text.strip()
    # Remove non-alphabetic characters (optional)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    text = ' '.join(word for word in word_tokens if word not in stop_words)
    return text

# Load the saved model
model = tf.keras.models.load_model('/content/sa_text (1).h5')

# Load the saved tokenizer
with open('/content/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load the saved max_length
with open('/content/max_length.pickle', 'rb') as handle:
    max_length = pickle.load(handle)

# Prompt the user for a raw input text
user_input = input("Enter a text to classify its emotion: ")

# Preprocess the input text using the normalization function
normalized_input = normalize(user_input)

# Convert the normalized text to a sequence using the trained tokenizer
input_sequence = tokenizer.texts_to_sequences([normalized_input])

# Pad the sequence to match the maximum sequence length used during training
input_padded = pad_sequences(input_sequence, maxlen=max_length)

# Use the model to predict the emotion
prediction = model.predict(input_padded)

# Get the predicted numeric label (the index of the highest probability)
predicted_numeric_label = np.argmax(prediction, axis=1)[0]

# Map the numeric label to the corresponding emotion name
predicted_emotion = label_mapping[predicted_numeric_label]
print("Predicted Emotion:", predicted_numeric_label)
print("Predicted Emotion:", predicted_emotion)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Enter a text to classify its emotion: feeling sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 921ms/step
Predicted Emotion: 0
Predicted Emotion: sadness


In [ ]:

# Define the mood mapping
mood_mapping = {
    "joy": 0,    # Happy
    "love": 0,   # Happy
    "sadness": 1, # Sad
    "fear": 1,   # Sad
    "anger": 1,  # Sad
    "surprise": 2 # Neutral
}

# Function to convert predicted_numeric_label to mood
def convert_to_mood(predicted_numeric_label):
    # Get the emotion from label_mapping
    emotion = label_mapping.get(predicted_numeric_label)
    # Map the emotion to mood
    mood = mood_mapping.get(emotion, 2)  # Default to 2 (neutral) if not found
    return mood

# Example usage
# predicted_numeric_label = 1  # Example: joy
mood = convert_to_mood(predicted_numeric_label)
print(f"Predicted numeric label: {predicted_numeric_label}")
print(f"Emotion: {label_mapping[predicted_numeric_label]}")
print(f"Mood: {mood}")  # Output: 0 (Happy)




Predicted numeric label: 3
Emotion: anger
Mood: 1


In [ ]:
import pickle
import pandas as pd
import numpy as np
with open('/content/dataset.pkl', 'rb') as f:
    df_filtered1 = pickle.load(f)
def recommend_movies(cluster_value):
    print('What do you want to see: Movie or TV series?')
    x=input()
    filtered_movies = df_filtered1[df_filtered1['cluster'] == cluster_value]
    if(x=='Movie'):
        top_movies = filtered_movies[filtered_movies['contentType']=='Movie'].head(10)
    else:
        top_movies = filtered_movies[filtered_movies['contentType']=='TV series'].head(10)

    top_movies = filtered_movies.head(10)  # Get the first 10 movies by index
    return top_movies[['dataId', 'title']]


print("it seems you are feeling", predicted_emotion)
print(f"Emotion: {label_mapping[predicted_numeric_label]}")
print(f"Mood: {mood}")  # Output: 0 (Happy)

# Example usage:0 - Happy,joy,love ; 1- Sad,fear,anger ; 2- neutral,disguist,surprise
recommended_movies = recommend_movies(mood)

results = recommended_movies
print(results)


it seems you are feeling sadness
Emotion: sadness
Mood: 1
What do you want to see: Movie or TV series?
movie
     dataId                        title
7    100007                        #Home
14   100014            C/o Kancharapalem
36   100036         Nuvvu Naaku Nachchav
134  100134    A Northern Story of Valor
125  100125            Thoovanathumbikal
29   100029                   Thalapathi
22   100022                   Drishyam 2
37   100037           Jaane Bhi Do Yaaro
46   100046                Chupke Chupke
51   100051  Agent Sai Srinivasa Athreya


In [ ]:
!pip install imdbpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import Image, display
from imdb import IMDb

def get_movie_info(title):
    try:
        # Initialize IMDbPy
        ia = IMDb()

        # Search for the movie
        movies = ia.search_movie(title)
        if not movies:
            print(f"No movie found for {title}")
            return None

        # Get the first movie result
        movie = movies[0]
        ia.update(movie)  # Fetch detailed info

        # Search for movie poster via Google Images
        query = title + ' poster'
        url = 'https://www.google.com/search?q=' + query + '&tbm=isch'
        headers = {'User-Agent': 'Mozilla/5.0'}
        content = requests.get(url, headers=headers).content
        soup = BeautifulSoup(content, 'html.parser')
        images = soup.find_all('img')

        # Display the poster (skip the first image if it's a logo)
        if len(images) > 1:
            display(Image(url=images[1].get('src')))
        else:
            print(f"Could not find poster for {title}")

        # Extract movie information
        print('Description:', movie.get('plot outline', 'N/A'))
        print('Release Year:', movie.get('year', 'N/A'))
        print('Rating:', movie.get('rating', 'N/A'))
        print('IMDb Link:', f"https://www.imdb.com/title/tt{movie.movieID}/")
        print('-' * 50)

        return {
            'title': title,
            'poster': images[1].get('src') if len(images) > 1 else None,
            'description': movie.get('plot outline', 'N/A'),
            'year': movie.get('year', 'N/A'),
            'rating': movie.get('rating', 'N/A'),
            'imdb_link': f"https://www.imdb.com/title/tt{movie.movieID}/"
        }

    except Exception as e:
        print(f"Error fetching info for {title}: {e}")
        return None

# Assuming 'results' is your DataFrame
results1 = results.reset_index(drop=True)
movie_data = []

# Loop through the first 9 titles
for i in range(min(9, len(results1))):
    print("Title:", results1['title'][i])
    info = get_movie_info(results1['title'][i])
    if info:
        movie_data.append(info)

# Convert to DataFrame for easy handling
movie_df = pd.DataFrame(movie_data)
print(movie_df)

Title: #Home
Could not find poster for #Home
Description: Light television comedy about family man Tim "The Tool Man" Taylor. The show's humor often revolved around cars, toys, tools, hardware shops, garages, fix-it-up projects, and similar themes.
Release Year: 1991
Rating: 7.2
IMDb Link: https://www.imdb.com/title/tt0101120/
--------------------------------------------------
Title: C/o Kancharapalem


Description: N/A
Release Year: 2018
Rating: 8.8
IMDb Link: https://www.imdb.com/title/tt7391996/
--------------------------------------------------
Title: Nuvvu Naaku Nachchav


Description: Venakteswarlu alias Venky (Venkatesh) comes to the house of Murthy (Prakash Raj) on recommendation of his father Sekhar (Chandra Mohan) who is a childhood friend of Murthy. Nandini alias Nandu (Arti Agarwal) is the daughter of Murthy. Venky enters that place when Nandu is about to get engaged to a software professional from the USA. Soon, Venky bonds with everyone from Murthy's family. Murthy helps Venky secure a job. The relationship between Venky and Nandu starts with hatred and turns into friendship. Nandu confesses her love for Venky to him. Venky, who does not want to spoil the relationship between their parents, decides to leave the house. But Murthy goes to the railway station and talks to Venky, which causes Venky to return. On his return, Nandu stops speaking to him. When Venky accompanies Nandu and Pinky (Nandu's cousin) to Nandu's friend Asha's wedding, sparks fly between them.Even though Venky also is in love with Nandu, he does not express it and pretends to n

Description: The film is set in 16th-century Northern Kerala. The plot unfolds at Puthooram, the house of great Kannappan Chekavar (Balan K. Nair). Kannappan Chekavar adopts the son of his estranged sister when the boy loses both his parents and brings him to Puthooram to live and learn with his cousins. The orphan boy, Chandu, a quick learner, earns the love and admiration of his uncle, while he is loathed by his cousin Aromal Chekavar (Suresh Gopi). As they grow up Chandu (Mammootty) is betrothed to Chekavar's daughter, Unniyarcha (Madhavi). Several events unfold where in Chandu finds himself being pushed to the corner by his cousin. He flees to the Kali under the guidance of Tulunadan expert and master Chekavar, Aringodar (Captain Raju).
Release Year: 1989
Rating: 8.7
IMDb Link: https://www.imdb.com/title/tt0230597/
--------------------------------------------------
Title: Thoovanathumbikal


Description: Jayakrishnan(Mohanlal), a bachelor guy, is leading a dual life - one in the town and the other with his mother and sister in the village. The story revolves around Thrissur, a place in Kerala. In the town he's a party man spending time with his friends but at home, he's a good family man. The story is about how he falls in love with two girls - Radha(Parvathy), one of his distant relatives in village and Clara(Sumalatha) a sex worker in town and then gets confused with the choice of whom he should spend his life with.
Release Year: 1987
Rating: 8.6
IMDb Link: https://www.imdb.com/title/tt0249188/
--------------------------------------------------
Title: Thalapathi


Description: Surya (Rajnikanth) is an orphan raised in a slum. He finds a friend in the local Godfather (Mamooty). They rule the town and forms a parallel government. Things are well until new district collect(Aravind Swami), step -brother of Surya takes charge. Surya finds his mom (Sreevidya) and decides to surrender.
Release Year: 1991
Rating: 8.5
IMDb Link: https://www.imdb.com/title/tt0101649/
--------------------------------------------------
Title: Drishyam 2


Description: A drug deal goes wrong between two peddlers where one named David accidentally shoots other while trying to escape he notices Vijay Salgaonkar coming out of under construction police station after hiding something.Seven years later Vijay lives happily with his family, along with his cable business he also owns a cinema hall and plans to produce his own film.Mahesh Deshmukh comes to meet Vijay asking him for final remains of his son but goes distressed on the other hand IG Meera Deshmukh has still not forgiven Vijay and waits for evidence.IG Tarun Alhawat colleague of Meera is posted sometime back in Goa and tries to re investigate Sam's case by getting suspended officer Gaitonde back on duty.Tarun and Meera lay a trap on Vijay and his family but very much far away from getting any clue.
Release Year: 2022
Rating: 8.2
IMDb Link: https://www.imdb.com/title/tt15501640/
--------------------------------------------------
Title: Jaane Bhi Do Yaaro


Description: Vinod (Naseeruddin Shah) and Sudhir (Ravi Baswani) are two professional photographers trying to start a photo studio. The first real assignment that they get, from the activist magazine Khabardar (literally meaning beware), draws them into the shady world of Bombay real estate. They discover that one of the biggest builders in Bombay, Tarneja, is trying to bribe Municipal Commissioner D'Mello (Satish Shah) into giving him a bridge contract. Meanwhile D'Mello is negotiating with Tarneja's arch rival Ahuja (Om Puri). The contract is finally awarded to a third person. By chance Vinod and Sudhir find out that Tarneja has murdered D'Mello...
Release Year: 1983
Rating: 8.3
IMDb Link: https://www.imdb.com/title/tt0085743/
--------------------------------------------------
Title: Chupke Chupke


Description: N/A
Release Year: 2021
Rating: 8.6
IMDb Link: https://www.imdb.com/title/tt15374748/
--------------------------------------------------
                       title  \
0                      #Home   
1          C/o Kancharapalem   
2       Nuvvu Naaku Nachchav   
3  A Northern Story of Valor   
4          Thoovanathumbikal   
5                 Thalapathi   
6                 Drishyam 2   
7         Jaane Bhi Do Yaaro   
8              Chupke Chupke   

                                              poster  \
0                                               None   
1  https://encrypted-tbn0.gstatic.com/images?q=tb...   
2  https://encrypted-tbn0.gstatic.com/images?q=tb...   
3  https://encrypted-tbn0.gstatic.com/images?q=tb...   
4  https://encrypted-tbn0.gstatic.com/images?q=tb...   
5  https://encrypted-tbn0.gstatic.com/images?q=tb...   
6  https://encrypted-tbn0.gstatic.com/images?q=tb...   
7  https://encrypted-tbn0.gstatic.com/images?q=tb...   
8  https://encrypted-tbn0.